In [ ]:
# Install required packages
!pip install easyocr
!pip install imutils

import cv2
import numpy as np
import imutils
import easyocr
from google.colab.patches import cv2_imshow
from google.colab import files

In [ ]:

def detect_license_plate(image_path):
    """
    Detect and recognize license plate from an image.

    Args:
        image_path (str): Path to the input image

    Returns:
        tuple: (processed image with detection, recognized text)
    """
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply filters and find edges
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is None:
        return img, "No license plate detected"

    # Create mask and perform bitwise operations
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)

    # Crop the license plate region
    (x, y) = np.where(mask == 255)
    (x1, y1) = (np.min(y), np.min(x))
    (x2, y2) = (np.max(y), np.max(x))
    cropped_image = gray[y1:y2+1, x1:x2+1]

    # Initialize EasyOCR
    reader = easyocr.Reader(['en'])

    # Read text from the license plate
    result = reader.readtext(cropped_image)
    text = ""

    # Draw rectangle and text
    if result:
        text = result[0][1]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return img, text

In [ ]:
# Usage
def process_uploaded_image():
    """Upload and process an image in Google Colab"""
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f'Processing {filename}...')

        # Process the image
        processed_img, plate_text = detect_license_plate(filename)

        # Display results
        print(f'Detected Text: {plate_text}')
        cv2_imshow(processed_img)

        # Option to download the processed image
        cv2.imwrite('processed_' + filename, processed_img)
        files.download('processed_' + filename)

# Run the pipeline
process_uploaded_image()